In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim

# Mount Google Drive (Jika perlu, aktifkan di Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Path ke dataset
DATASET_PATH = "/content/drive/MyDrive/COMVIS/Kelompok/dataset"  # Sesuaikan dengan lokasi dataset Anda


Mounted at /content/drive


In [ ]:
# Data Transformasi dengan Augmentasi
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load dataset
dataset = ImageFolder(root=DATASET_PATH, transform=data_transforms)

# Split dataset menjadi Train, Validation, Test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load Pretrained ResNet18
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Sesuaikan output layer dengan jumlah kelas dataset

# Freeze semua layer kecuali FC layer
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

# Inisialisasi Loss dan Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

    # Evaluasi di Validation Set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total:.2f}%")


Epoch 1/10, Loss: 1.4629
Validation Accuracy: 62.80%
Epoch 2/10, Loss: 1.0398
Validation Accuracy: 68.60%
Epoch 3/10, Loss: 0.8954
Validation Accuracy: 73.61%
Epoch 4/10, Loss: 0.8061
Validation Accuracy: 68.07%
Epoch 5/10, Loss: 0.7757
Validation Accuracy: 72.82%
Epoch 6/10, Loss: 0.7022
Validation Accuracy: 79.16%
Epoch 7/10, Loss: 0.6999
Validation Accuracy: 73.09%
Epoch 8/10, Loss: 0.6998
Validation Accuracy: 74.14%
Epoch 9/10, Loss: 0.7200
Validation Accuracy: 78.36%
Epoch 10/10, Loss: 0.7027
Validation Accuracy: 78.36%


In [ ]:
# Simpan Model
torch.save(model.state_dict(), "klasifikasiSampah.pth")
print("Model saved successfully!")

Model saved successfully!
